In [ ]:
import joblib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Patch

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette(['#6da7de', '#9e0059', '#dee000', '#d82222', '#5ea15d',
                 '#943fa6', '#63c5b5', '#ff38ba', '#eb861e', '#ee266d'])
sns.set_context('paper', font_scale=1.3)    # Single-column figure.

In [ ]:
hyperparam = pd.read_csv('cluster_comparison_hyperparam.csv')
cluster_sizes = joblib.load('cluster_comparison_size.joblib')
max_peptide_clusters_grouped = pd.read_csv('cluster_comparison_size.csv')

In [ ]:
tools = ('falcon', 'MaRaCluster', 'MS-Cluster', 'msCRUSH', 'spectra-cluster')

In [ ]:
width = 7
height = width / 1.618
fig, axes = plt.subplots(2, 2, figsize=(width * 2, height * 2))
axes = np.ravel(axes)

# Number of clustered spectra and completeness.
for tool in tools:
    tool_performance = (hyperparam[(hyperparam['tool'] == tool) &
                                   (hyperparam['min_cluster_size'] == 2)]
                        .sort_values('prop_clustered_incorrect'))
    axes[0].plot(tool_performance['prop_clustered_incorrect'],
                 tool_performance['prop_clustered'], marker='o', label=tool)
    axes[1].plot(tool_performance['prop_clustered_incorrect'],
                 tool_performance['completeness'], marker='o', label=tool)

axes[0].set_xlim(0, 0.05)
axes[0].set_ylim(0, 1)
axes[1].set_xlim(0, 0.05)
axes[1].set_ylim(0.75, 1)

axes[0].xaxis.set_major_formatter(mticker.PercentFormatter(1, 0))
axes[0].yaxis.set_major_formatter(mticker.PercentFormatter(1, 0))
axes[1].xaxis.set_major_formatter(mticker.PercentFormatter(1, 0))

axes[0].set_xlabel('Incorrectly clustered spectra')
axes[0].set_ylabel('Clustered spectra')
axes[1].set_xlabel('Incorrectly clustered spectra')
axes[1].set_ylabel('Completeness')

axes[0].legend(loc='lower right', frameon=False)
axes[1].legend(loc='lower right', frameon=False)

# Cluster sizes.
for i, tool in enumerate(tools):
    sns.ecdfplot(cluster_sizes[tool], stat='proportion', ax=axes[2],
                 label=tool, zorder=len(tools) - i)
    
axes[2].set_xscale('log')
axes[2].set_ylim(0., 1.01)

axes[2].yaxis.set_major_formatter(mticker.PercentFormatter(1))

axes[2].set_xlabel('Cluster size')
axes[2].set_ylabel('Cumulative number of spectra')

axes[2].legend(loc='lower right', frameon=False)

# Frequent peptide.
for position, tool, hatch in zip(
        [2.5, 1.5, 0.5, -0.5, -1.5], tools, ['x', '', '/', '.', '\\']):
    (max_peptide_clusters_grouped[max_peptide_clusters_grouped['tool'] == tool]
     .plot.bar(x='interval', position=position, rot=0, stacked=True,
               width=0.15, edgecolor='black', hatch=3 * hatch, ax=axes[3]))
    
# Expand the axis so that the legend fits.
box = axes[3].get_position()
axes[3].set_position([box.x0, box.y0, box.width, box.height * 1.05])

legend_elements = [
    Patch(facecolor='#6da7de', edgecolor='black',label='VATVSIPR/2'),
    Patch(facecolor='white', edgecolor='black', hatch=3 * 'x',
          label='falcon'),
    Patch(facecolor='white', edgecolor='black', hatch=3 * '.',
          label='msCRUSH'),
    Patch(facecolor='#9e0059', edgecolor='black', label='Unidentified'),
    Patch(facecolor='white', edgecolor='black', hatch=3 * '',
          label='MaRaCluster'),
    Patch(facecolor='white', edgecolor='black', hatch=3 * '\\',
          label='spectra-cluster'),
    Patch(facecolor='#dee000', edgecolor='black', label='Incorrect peptide'),
    Patch(facecolor='white', edgecolor='black', hatch=3 * '/',
          label='MS-Cluster')]
axes[3].legend(handles=legend_elements, loc='center',
               bbox_to_anchor=(0.5, 1.15), ncol=3, frameon=False)

axes[3].set_xlim(-0.5, axes[3].get_xlim()[1])

axes[3].yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}'))

axes[3].set_xlabel('Cluster size')
axes[3].set_ylabel('Clustered spectra')

for i, (ax, pos, c) in enumerate(zip(
        axes, [(0.03, 0.92), (0.5, 0.92), (0.03, 0.47), (0.5, 0.47)], 'ABCD')):
    ax.annotate(c, xy=pos, xycoords='figure fraction',
                fontsize='xx-large', weight='bold')

# for i, (ax, c) in enumerate(zip(axes, 'ABCD')):
#     ax.annotate(c, xy=(-0.15, 1.1), xycoords='axes fraction',
#                 fontsize='xx-large', weight='bold')

for ax in axes:
    sns.despine(ax=ax)
    
fig.tight_layout()

plt.savefig('cluster_comparison.pdf', dpi=300, bbox_inches='tight',
            pad_inches=0.15)
plt.show()
plt.close()